## Load data

In [ ]:
import geopandas as gpd

canadian_terretories = gpd.read_file("./data/lpr_000b16a_e/lpr_000b16a_e.shp")
nwt = canadian_terretories[canadian_terretories.PREABBR == "N.W.T."]
banks_island = nwt.explode().reset_index()[2:3]
del nwt, canadian_terretories
banks_island.explore()

In [ ]:
from odc.geo.geobox import GeoBox, Resolution

aoi = GeoBox.from_bbox(banks_island.to_crs("EPSG:3413").total_bounds, resolution=Resolution(32, -32), crs="EPSG:3413")
aoi

In [ ]:
import geopandas as gpd

rts = gpd.read_file("./data/DARTS_NitzeEtAl_v1_features_2023_BANKS_level2_enriched.gpkg")
rts

In [ ]:
import odc.geo.xr
from smart_geocubes.datasets import ArcticDEM32m

adem_accessor = ArcticDEM32m("./data/arcticdem32m.icechunk")
try:
    adem_accessor.create()
except FileExistsError:
    pass

adem = adem_accessor.load(aoi)

# Crop to banks island
geom = odc.geo.geom.Geometry(banks_island.to_crs("EPSG:3413").iloc[0].geometry, crs="EPSG:3413")
rasterized = odc.geo.xr.rasterize(geom, adem.odc.geobox)
adem = adem.where(rasterized)
del rasterized, geom

adem

In [ ]:
from folium import LayerControl

m = rts.explore(column="area_m2", name="rts", tiles="CartoDB positron")
adem.dem.odc.explore(map=m, cmap="gray")
LayerControl().add_to(m)
m

In [ ]:
import seaborn as sns

sns.jointplot(data=rts, x="area_m2", y="avg_elevation", kind="hex")

## Complete Spatial Randomness

In [ ]:
import numpy as np
import scipy.stats as stats
import xarray as xr
import polars as pl
import hvplot.polars


def quad_count(gdf: gpd.GeoDataFrame, nbins: int) -> xr.DataArray:
    """Rasterize a GeoDataFrame with quad-counting.

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame to rasterize.
        nbins (int): Number of bins in each dimension.

    Returns:
        xr.DataArray: Rasterized data.

    """
    counts, xedges, yedges = np.histogram2d(gdf.geometry.x, gdf.geometry.y, bins=nbins)
    x_coords = (xedges[1:] + xedges[:-1]) / 2
    y_coords = (yedges[1:] + yedges[:-1]) / 2
    xcounts = xr.DataArray(counts, coords={"x": x_coords, "y": y_coords}, dims=["x", "y"], name="counts")
    xcounts = xcounts.odc.assign_crs(gdf.crs)
    xcounts = xcounts.transpose("y", "x")
    return xcounts


def quad_test(gdf: gpd.GeoDataFrame, nbins: int) -> tuple[float, float]:
    """Test quad-counting.

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame to rasterize.
        nbins (int): Number of bins in each dimension.

    Returns:
        float: Test statistic.

    """
    counts = quad_count(gdf, nbins)
    estc = len(gdf) / counts.size
    pearson_residuals = (counts - estc) / np.sqrt(estc)
    chi_squared = (pearson_residuals**2 / estc).sum().item()
    # expected_chi_squared = stats.chi2.isf(0.05, df=counts.size - 1)
    p_value = stats.chi2.sf(chi_squared, df=counts.size - 1)
    return chi_squared, p_value


# Turn rts into points
rts_points = rts.set_geometry(rts.geometry.centroid)

quad_tests = pl.DataFrame(
    [[int(n_), *quad_test(rts_points, int(n_))] for n_ in np.arange(2, 10, 1)],
    orient="row",
    schema=["n", "chi2", "p_value"],
)
quad_tests.hvplot(x="n", y="p_value", title="Quad-counting test")

## Variogram

In [ ]:
import gstools as gs

bins = np.arange(0, 400, 10)
bin_center, gamma = gs.vario_estimate((rts_points.geometry.x, rts_points.geometry.y), rts_points["avg_elevation"])
fit_model = gs.Exponential(dim=2)
# Manual experiments showed nugget has very low effect on the variogram - hence we keep the default
fit_model.fit_variogram(bin_center, gamma)
fit_model.plot(x_max=bin_center.max()).scatter(bin_center, gamma, alpha=0.5);

In [ ]:
import matplotlib.pyplot as plt

models = {
    "Gaussian": gs.Gaussian,
    "Exponential": gs.Exponential,
    "Matern": gs.Matern,
    "Stable": gs.Stable,
    "Rational": gs.Rational,
    "Circular": gs.Circular,
    "Spherical": gs.Spherical,
    "SuperSpherical": gs.SuperSpherical,
    "JBessel": gs.JBessel,
}
scores = {}

# plot the estimated variogram
plt.scatter(bin_center, gamma, color="k", label="data", alpha=0.5)
ax = plt.gca()

# fit all models to the estimated variogram
for model in models:
    fit_model = models[model](dim=2)
    try:
        para, pcov, r2 = fit_model.fit_variogram(bin_center, gamma, return_r2=True)
    except Exception as e:
        print(f"Failed to fit {model}: {e}")
        continue
    fit_model.plot(x_max=bin_center.max(), ax=ax)
    scores[model] = r2

ranking = sorted(scores.items(), key=lambda item: item[1], reverse=True)
print("RANKING by Pseudo-r2 score")
for i, (model, score) in enumerate(ranking, 1):
    print(f"{i:>6}. {model:>15}: {score:.5}")

plt.show();

## Kriging

In [ ]:
model = gs.SuperSpherical(dim=2)
para, pcov = model.fit_variogram(bin_center, gamma)
ok2 = gs.krige.Ordinary(model, (rts_points.geometry.x, rts_points.geometry.y), rts_points["avg_elevation"])
ok2

In [ ]:
grid_x = np.arange(aoi.boundingbox.left, aoi.boundingbox.right, 640)
grid_y = np.arange(aoi.boundingbox.top, aoi.boundingbox.bottom, -640)
ok2.structured([grid_x, grid_y])
# ok2.plot()